In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('SWaT_dataset_Jul_19_v2.csv')

/Users/guillaume/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,74,75,76,77) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df.head()

,Unnamed: 0,P1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,P2,Unnamed: 8,Unnamed: 9,...,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77
0,GMT +0,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,LSH 601,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status
1,timestamp,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
2,2019-07-20T04:30:00Z,0,729.8658,1,3,2,1,142.527557,9.293002,198.077423,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
3,2019-07-20T04:30:01Z,0,729.434,1,3,2,1,142.527557,9.293002,198.385025,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
4,2019-07-20T04:30:02.004013Z,0,729.12,1,3,2,1,142.527557,9.293002,198.4363,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1


On met les bons noms de colones

In [5]:
df.columns = df.loc[0]

In [6]:
df.head()

,GMT +0,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,LSH 601,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status
0,GMT +0,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,LSH 601,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status
1,timestamp,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
2,2019-07-20T04:30:00Z,0,729.8658,1,3,2,1,142.527557,9.293002,198.077423,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
3,2019-07-20T04:30:01Z,0,729.434,1,3,2,1,142.527557,9.293002,198.385025,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
4,2019-07-20T04:30:02.004013Z,0,729.12,1,3,2,1,142.527557,9.293002,198.4363,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1


On supprime les lignes inutiles

In [8]:
df = df.drop([0,1], axis=0)

In [9]:
df.head()

,GMT +0,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,LSH 601,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status
2,2019-07-20T04:30:00Z,0,729.8658,1,3,2,1,142.527557,9.293002,198.077423,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
3,2019-07-20T04:30:01Z,0,729.434,1,3,2,1,142.527557,9.293002,198.385025,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
4,2019-07-20T04:30:02.004013Z,0,729.12,1,3,2,1,142.527557,9.293002,198.4363,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
5,2019-07-20T04:30:03.004013Z,0,728.6882,1,3,2,1,142.527557,9.289157,198.667,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
6,2019-07-20T04:30:04Z,0,727.7069,1,3,2,1,142.527557,9.289157,198.89772,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1


Mise a jour des index

In [10]:
df.reset_index(inplace=True, drop=True)

In [11]:
df.head()

,GMT +0,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,LSH 601,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status
0,2019-07-20T04:30:00Z,0,729.8658,1,3,2,1,142.527557,9.293002,198.077423,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
1,2019-07-20T04:30:01Z,0,729.434,1,3,2,1,142.527557,9.293002,198.385025,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
2,2019-07-20T04:30:02.004013Z,0,729.12,1,3,2,1,142.527557,9.293002,198.4363,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
3,2019-07-20T04:30:03.004013Z,0,728.6882,1,3,2,1,142.527557,9.289157,198.667,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
4,2019-07-20T04:30:04Z,0,727.7069,1,3,2,1,142.527557,9.289157,198.89772,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1


On modifie le nom de la colone temps

In [13]:
df = df.rename(columns={"GMT +0":"timestamp"})

<class 'str'>


On modifie cette colone en ne gardant que l'heure, les minutes et les secondes et on stocke sous format timedelta

!! A voir si il faut pas garder aussi les millisecondes et les micro secondes !!

In [14]:
import re
from datetime import timedelta

In [15]:
date_pattern = '[0-9]{2}:[0-9]{2}:[0-9]*'
i=0
for date in df['timestamp']:
    hour = re.findall(date_pattern, date)
    h = str(hour)[2:10]
    n = h.split(':')
    df['timestamp'][i] = timedelta(hours=int(n[0]), minutes=int(n[1]), seconds=int(n[2]))
    i+=1

In [16]:
df.head()

,timestamp,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,LSH 601,LSH 602,LSH 603,LSL 601,LSL 602,LSL 603,P6 STATE,P601 Status,P602 Status,P603 Status
0,4:30:00,0,729.8658,1,3,2,1,142.527557,9.293002,198.077423,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
1,4:30:01,0,729.434,1,3,2,1,142.527557,9.293002,198.385025,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
2,4:30:02,0,729.12,1,3,2,1,142.527557,9.293002,198.4363,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
3,4:30:03,0,728.6882,1,3,2,1,142.527557,9.289157,198.667,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1
4,4:30:04,0,727.7069,1,3,2,1,142.527557,9.289157,198.89772,...,Active,Active,Inactive,Inactive,Inactive,Active,2,1,1,1


On creer les colones qui serviront pour labeliser les attaques
Pour l'instant que des 0

In [17]:
df['FIT401_attack'] = pd.Series(np.zeros(len(df)).astype(int))
df['LIT301_attack'] = pd.Series(np.zeros(len(df)).astype(int))
df['P601_attack'] = pd.Series(np.zeros(len(df)).astype(int))
df['MultiPoint_attack'] = pd.Series(np.zeros(len(df)).astype(int))
df['MV501_attack'] = pd.Series(np.zeros(len(df)).astype(int))
df['P301_attack'] = pd.Series(np.zeros(len(df)).astype(int))

In [18]:
df.head()

,timestamp,FIT 101,LIT 101,MV 101,P1_STATE,P101 Status,P102 Status,AIT 201,AIT 202,AIT 203,...,P6 STATE,P601 Status,P602 Status,P603 Status,FIT401_attack,LIT301_attack,P601_attack,MultiPoint_attack,MV501_attack,P301_attack
0,4:30:00,0,729.8658,1,3,2,1,142.527557,9.293002,198.077423,...,2,1,1,1,0,0,0,0,0,0
1,4:30:01,0,729.434,1,3,2,1,142.527557,9.293002,198.385025,...,2,1,1,1,0,0,0,0,0,0
2,4:30:02,0,729.12,1,3,2,1,142.527557,9.293002,198.4363,...,2,1,1,1,0,0,0,0,0,0
3,4:30:03,0,728.6882,1,3,2,1,142.527557,9.289157,198.667,...,2,1,1,1,0,0,0,0,0,0
4,4:30:04,0,727.7069,1,3,2,1,142.527557,9.289157,198.89772,...,2,1,1,1,0,0,0,0,0,0


# A faire :

## - Labeliser les données representant les 6 attaques
## - Etudier les données comme dans tp1 et tp2 (min, max, ...)
## - Appliquer un premier algo d'analyse